In [2]:
import os
import logging

import psycopg2
import psycopg2.extensions
from pymongo import MongoClient
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, Float, MetaData, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [107]:
import numpy as np

In [3]:
#обращение к PSQL в виртуалке через проброс порта
db_params = {
    "host":"127.0.0.1",
    "port": 5432,
    "dbname": "netology",
    "user": "user_1",
    "password": "test123"
}

In [11]:
conn = psycopg2.connect(**db_params)

# дополнительные настройки
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()

In [14]:
sql_1 = (
    'select * into movies_top from (select movieId, count(rating) as ratings_num, avg(rating) as ratings_avg '
    'from ratings group by movieId having avg(rating) > 3) as _top'
        )

In [15]:
cursor.execute(sql_1)
conn.commit()

In [17]:
# Проверка - выгружаем данные
cursor.execute("SELECT * FROM movies_top LIMIT 10")
[i for i in cursor.fetchall()]

[(790, 6, 3.16666666666667),
 (146688, 2, 3.25),
 (69906, 1, 5.0),
 (26745, 5, 3.3),
 (3936, 18, 3.44444444444444),
 (88837, 3, 3.16666666666667),
 (31297, 1, 4.5),
 (5468, 6, 3.75),
 (49200, 3, 3.33333333333333),
 (155078, 1, 4.5)]

In [18]:
# Задание по SQLAlchemy
# --------------------------------------------------------------
Base = declarative_base()


class MoviesTop(Base):
    __tablename__ = 'movies_top'

    movieid = Column(Integer, primary_key=True)
    ratings_num = Column(Float)
    ratings_avg = Column(Float)

    def __repr__(self):
        return "<User(movieid='%s', ratings_num='%s', ratings_avg='%s')>" % (self.movieid, self.ratings_num, self.ratings_avg)

In [24]:
# Создаём сессию

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(db_params["user"], db_params["password"],
                                                         db_params["host"], db_params["port"], db_params["dbname"]))
Session = sessionmaker(bind=engine)
session = Session()

In [45]:
sa_query = session.query(MoviesTop.movieid).filter(MoviesTop.ratings_num>15).filter(MoviesTop.ratings_avg>3.5).order_by(MoviesTop.ratings_avg)

In [108]:
top_ids = np.array(sa_query.all())

In [111]:
top_ids = top_ids.flatten()

In [53]:
# Прочитаем теги из psql, разберем и положим в mongo

In [55]:
cursor.execute("SELECT * FROM keywords")
kwords =  cursor.fetchall()

In [73]:
tags_data = []

In [77]:
for l in kwords:
    tags = eval(l[1])
    for tag in tags:
        tag.update({'movieId': l[0]})
        tags_data.append(tag)

In [78]:
len(tags_data)

158680

In [87]:
tags_data[:10]

[{'id': 931, 'name': 'jealousy', 'movieId': 862},
 {'id': 4290, 'name': 'toy', 'movieId': 862},
 {'id': 5202, 'name': 'boy', 'movieId': 862},
 {'id': 6054, 'name': 'friendship', 'movieId': 862},
 {'id': 9713, 'name': 'friends', 'movieId': 862},
 {'id': 9823, 'name': 'rivalry', 'movieId': 862},
 {'id': 165503, 'name': 'boy next door', 'movieId': 862},
 {'id': 170722, 'name': 'new toy', 'movieId': 862},
 {'id': 187065, 'name': 'toy comes to life', 'movieId': 862},
 {'id': 10090, 'name': 'board game', 'movieId': 8844}]

In [88]:
mongo_params = {
    'host': 'localhost',
    'port': 27017
}

mongo = MongoClient(**mongo_params)

In [92]:
mongo_db = mongo.get_database(name = "test_db")

In [93]:
#положили один раз и закомментировали

#mongo_coll = mongo_db.tags_collection
#mongo_coll.insert_many(tags_data)

In [124]:
mongo_query = mongo_db.tags_collection.find({'movieId': {'$in':top_ids.tolist()}})

In [125]:
df =  pd.DataFrame(list(mongo_query))

In [130]:
df.head()

,_id,id,movieId,name
0,5c390e2851a9d431f8f76236,642,949,robbery
1,5c390e2851a9d431f8f76237,703,949,detective
2,5c390e2851a9d431f8f76238,974,949,bank
3,5c390e2851a9d431f8f76239,1523,949,obsession
4,5c390e2851a9d431f8f7623a,3713,949,chase


In [141]:
df.groupby(by="name").count().sort_values(by='movieId',ascending=False).head(5)

,_id,id,movieId
name,,,
murder,39,39,39
independent film,38,38,38
friendship,33,33,33
suspense,29,29,29
dying and death,29,29,29
